# Automated Infusion Evaluation System

This notebook automates the process of running multiple trials for different model configurations (baseline, infused-fixed, infused-trainable) to evaluate and compare their performance systematically. It collects validation metrics for each run, aggregates them, and saves the results.


### ||RUN ON RESTART||

In [ ]:
# Load dependencies

from utils import build_multilabel_dataset, multilabel_split, prep_infused_sweetnet, seed_everything

import os
import pickle

from glycowork.ml.processing import split_data_to_train
from glycowork.ml import model_training


In [ ]:
# Load embeddings

pickle_file_path = 'glm_embeddings_1.pkl'

# --- Load the Pickle File ---
if os.path.exists(pickle_file_path):
    print(f"Loading embeddings from: {pickle_file_path}")
    try:
        # Open the file in binary read mode ('rb')
        with open(pickle_file_path, 'rb') as file_handle:
            # Load the object(s) from the pickle file
            glm_embeddings = pickle.load(file_handle)

        print("Embeddings loaded successfully!")        

    except Exception as e:
        print(f"An error occurred while loading the pickle file: {e}")
else:
    print(f"Error: File not found at '{pickle_file_path}'. Please check the filename and path.")

## Evaluation Loop
Change parameters here for each trial run.


In [ ]:
# Load part of dataset to train the model on

glycans, labels, label_names = build_multilabel_dataset(glycan_dataset='df_species', 
                                                        glycan_class='Kingdom', 
                                                        min_class_size=6)

In [ ]:
# --- Global Parameters & Experiment Configuration ---
NUM_RUNS = 10  # Number of trials per configuration (e.g., 5 or 10)
EPOCHS = 100 # Number of training epochs per run
BATCH_SIZE = 128
TRAIN_SIZE = 0.7 # Fraction of data to use for training (0.7 = 70% train, 15% val, 15% test)
LEARNING_RATE = 0.005 # Learning rate for the optimizer
DROP_LAST = False, # Whether to drop the last batch if it's smaller than the batch size
AUGMENT_PROB = 0.0,  # Adjust if you want augmentation for training
GENERALIZATION_PROB = 0.2  # Adjust if you want generalization for training
BASE_RANDOM_STATE = 42 # Initial seed for reproducibility of the entire experiment sequence
RESULTS_SUMMARY_CSV_PATH = "automated_evaluation_summary.csv"
ALL_EPOCH_DATA_PKL_PATH = "automated_all_epoch_data.pkl"

# Set the random seed for reproducibility
seed_everything(BASE_RANDOM_STATE)

# Initialize lists/dicts to store results globally for the notebook session
all_run_summary_results = [] # For storing best metrics from each run
all_run_epoch_histories = {} # For storing full epoch-wise histories

In [ ]:
# Run settings

# file to save the run data to
saved_run_data = "evaluation_run_dump"

trial_seed = 2
#increment each trial by 1

config_description = 'Kindomd'
# baseline, infused_train, or infused

learning_rate = 0.005

In [ ]:
# Split the dataset into training, validation, and test sets
train_glycans, val_glycans, test_glycans, \
    train_labels, val_labels, test_labels = multilabel_split(glycans, labels, train_size=0.7, 
                                                             random_state=trial_seed)

# Load into dataloders for training and validation
dataloaders = split_data_to_train(
    glycan_list_train = train_glycans, glycan_list_val = val_glycans, labels_train = train_labels, labels_val = val_labels,
    batch_size = 128,  # 32 or 128 seem to work well on this system
    drop_last = False,
    augment_prob = 0.0,  # Adjust if you want augmentation for training
    generalization_prob = 0.2  # Adjust if you want generalization for training
)

In [ ]:
# model training 

classes = len(labels[0]) # number of classes in the dataset

model =  prep_infused_sweetnet(
            initialization_method = 'external', # random or external
            num_classes = classes,
            embeddings_dict = glm_embeddings, 
            trainable_embeddings = True, # True or False
            ) 

optimizer_ft, scheduler, criterion = model_training.training_setup(model, learning_rate, num_classes = classes)

model_ft, current_run_metrics = model_training.train_model(model, dataloaders, criterion, optimizer_ft, scheduler,
                   num_epochs = 100, mode = 'multilabel', return_metrics = True)

run_identifier = f"{config_description}_{trial_seed}"
all_training_histories[run_identifier] = current_run_metrics

saved_run_data_path = (f"{saved_run_data}.pkl")

# Save the entire collection at the end (or periodically)
with open(saved_run_data_path, 'wb') as f:
    pickle.dump(all_training_histories, f)
print(f"Saved training histories to {saved_run_data_path}")

In [ ]:
print(all_training_histories)

In [ ]:
# Load trial data

pickle_file_path = 'evaluation_run_1.pkl'

# --- Load the Pickle File ---
if os.path.exists(pickle_file_path):
    print(f"Loading data from: {pickle_file_path}")
    try:
        # Open the file in binary read mode ('rb')
        with open(pickle_file_path, 'rb') as file_handle:
            # Load the object(s) from the pickle file
            user_data_string_from_input = pickle.load(file_handle)

        print("Data loaded successfully!")        

    except Exception as e:
        print(f"An error occurred while loading the pickle file: {e}")
else:
    print(f"Error: File not found at '{pickle_file_path}'. Please check the filename and path.")

In [ ]:
print(user_data_string_from_input)